# This file is to take the .xlsx file created by reach_Curator_py38_v3.py script, and adds in the missing rows for T6000, T5000 values
- has multiple steps
1. Backs up the raw .xlsx file created by Reach_Curator_py38_v3.py
2. Takes raw .xlsx file created by Reach_Curator_py38_v3.py --> Adds in missing rows by reading _events.txt file and comparing it --> puts this modifed version with added rows in the root folder Grant_curate
3. Once you have manually curated the session (ie. added reaches) using the Reach_Curator_py38_v3.py
4. this will back up that manually curated .xlsx file into this folder, which it creates if it does not exist already --> completed_manual_curation_backup
5. finally, it will remove any rows that have no values for ReachInit > ReachMax > ReachEnd_ --> it will then put this final cleaned version back into the root folder that the Reach_Curator_py38_v3.py saves and reads from --> Grant_curate

In [122]:
import pandas as pd
import numpy as np
import os
import shutil


## Modify these three variables as needed

In [123]:
root_path = r'G:\Grant\behavior_data\DLC_net' # change this to your root path where all the analysis folder live, i currently use is G:\Grant\behavior_data\DLC_net
root_folder = 'grant_reach10_swingDoor-christie-2025-04-21' # change this to your root folder for one specfic mouse (this should contain multiple sessions)

# Set the specific mouse session to analyze
session_info = '20250421_session001'

### Create new session_option for each session
- class that lets you load data for specific mouse sesssion

In [124]:
main_path = rf'{root_path}\{root_folder}'
print(f'main_path: {main_path}')

class select_mouse_session:
    def __init__(self, mouse, session_date, session_ID):
        self.mouse = mouse
        self.session_date = session_date
        self.session_ID = session_ID

    def __repr__(self):
        return f"Mouse: {self.mouse}, Session Date: {self.session_date}, Session ID: {self.session_ID}"
    
    def get_mouse_session_files(self):
        mouse_session = f"{self.mouse}_{self.session_date}_{self.session_ID}"
        xlsx_file = rf'{main_path}\Grant_curate\{self.session_date}_christielab_{self.session_ID}.xlsx'
        txt_file = r"G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\videos\20250421\christielab\session001\20250421_christielab_session001_events_shifted.txt"
        xlsx_orig = rf'G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\curator_direct_backup\{self.session_date}_christielab_{self.session_ID}.xlsx'
        
        return xlsx_file, txt_file, xlsx_orig
    
    def get_mouse_info(self):
        mouse_info = {
            'mouse': self.mouse,
            'session_date': self.session_date,
            'session_ID': self.session_ID
        }
        return mouse_info



main_path: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21


In [125]:
print(f"⚠️YOU ARE ABOUT TO MODIFY --> {session_info}⚠️")
print('')

if session_info == '20250421_session001':
    # create a new class instance
    reach10_20250421_session01 = select_mouse_session(mouse='reach10', session_date='20250421', session_ID='session001')
    xlsx_file, txt_file, xlsx_orig = reach10_20250421_session01.get_mouse_session_files()
    mouse_info = reach10_20250421_session01.get_mouse_info()
    print(f'mouse_info: {mouse_info}')
    print(f'xlsx_file: {xlsx_file}')

if session_info == '20250424_session001':
    # create a new class instance
    reach10_20250425_session01 = select_mouse_session(mouse='reach10', session_date='20250424', session_ID='session001')
    xlsx_file, txt_file, xlsx_orig = reach10_20250425_session01.get_mouse_session_files()
    mouse_info = reach10_20250425_session01.get_mouse_info()
    print(f'mouse_info: {mouse_info}')
    print(f'xlsx_file: {xlsx_file}')



    # ---- Step 1: Make a backup in curator_backup folder
original_dir = os.path.dirname(xlsx_file)
backup_dir = os.path.join(original_dir, 'curator_direct_backup')
os.makedirs(backup_dir, exist_ok=True)
print(f'backup_dir: {backup_dir}')
print(f'original_dir: {original_dir}')

⚠️YOU ARE ABOUT TO MODIFY --> 20250421_session001⚠️

mouse_info: {'mouse': 'reach10', 'session_date': '20250421', 'session_ID': 'session001'}
xlsx_file: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250421_christielab_session001.xlsx
backup_dir: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\curator_direct_backup
original_dir: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate


## Verify existence of session files

In [133]:
if not os.path.exists(txt_file):
    print(f"⚠️Warning: {txt_file} does not exist⚠️")

backup_file_path = os.path.join(backup_dir, os.path.basename(xlsx_file))
if os.path.exists(backup_file_path):
    print(f"⚠️Warning: Backup file already exists at {backup_file_path}⚠️")
else:
        # Check if files exist
    if not os.path.exists(xlsx_file):
        print(f"⚠️Warning: {xlsx_file} does not exist⚠️")
    else:
        # Copy the original file to the backup directory
        shutil.copy2(xlsx_file, backup_file_path)
        print(f"✅ Backup saved to: {backup_file_path}")

# ---- (your parsing & merging code goes here)
# ... your code to build `combined` DataFrame ...

print(f'backup_dir: {backup_file_path}')


✅ Backup saved to: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\curator_direct_backup\20250421_christielab_session001.xlsx
backup_dir: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\curator_direct_backup\20250421_christielab_session001.xlsx


### This is the cell that takes the .xlsx file and the .txt file and creates a new .xlsx file with the manual additions
- makes a new folder called manual_additions if it does not exist
- reads the .xlsx file into a pandas dataframe
- reads the .txt file into a pandas dataframe
- creates a new dataframe with the manual additions
- saves the new dataframe to a new .xlsx file



In [134]:
# Load existing curated xlsx
print(f"Loading existing xlsx file: {xlsx_file}")
df_orig = pd.read_excel(xlsx_file)
df_orig.head()

Loading existing xlsx file: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250421_christielab_session001.xlsx


,T6000,T5000,reachInit,reachMax,reachEnd,stim,behaviors,pellet_delivery,pellet_detected
0,1768,2205.0,NaN,NaN,NaN,0,NaN,2203.0,1823.0
1,4255,4388.0,4423.0,4436.0,4449.0,0,grabbed,4385.0,4308.0
2,6582,6833.0,NaN,NaN,NaN,0,NaN,6831.0,6642.0
3,8895,9263.0,NaN,NaN,NaN,0,NaN,9261.0,8956.0
4,13422,13632.0,13669.0,13681.0,13696.0,0,grabbed,13630.0,13477.0


In [135]:

class modify_curator_xlsx_file(object):
    def __init__(self, xlsx_file, txt_file,save_df=True):
        self.txt_file = txt_file
        self.xlsx_file = xlsx_file
        self.save_df = save_df

        
    def create_new_xlxs_file(self):
        save_df = self.save_df
        # Load the existing xlsx file
        df_orig = pd.read_excel(self.xlsx_file)
        # Check if the file is empty
        txt_file = self.txt_file
        if not os.path.exists(txt_file):
            print(f"⚠️Warning: {txt_file} does not exist⚠️")
            return None
        # Check if the xlsx file is empty
        xlsx_file = self.xlsx_file
        if not os.path.exists(xlsx_file):
            print(f"⚠️Warning: {xlsx_file} does not exist⚠️")
            return None

        if df_orig.empty:
            print("⚠️Warning: The xlsx file is empty⚠️")
            return None
        
        # Normalize column names just in case
        df_orig.columns = df_orig.columns.str.strip()

        # Load and parse txt as done earlier
        events = []
        current_event = {}
        with open(txt_file, 'r') as f:
            for line in f:
                if line.strip():
                    key, value = line.strip().split('\t')
                    value = int(value)
                    if key == 'T6000_played':
                        if current_event:
                            events.append(current_event)
                        current_event = {'T6000': value}
                    elif key == 'pellet_detected':
                        current_event['pellet_detected'] = value
                    elif key == 'pellet_delivery':
                        current_event['pellet_delivery'] = value
                    elif key == 'T5000_played':
                        current_event['T5000'] = value
        if current_event:
            events.append(current_event)

        events_df = pd.DataFrame(events)

        # Find which T6000s are missing from original file
        existing_T6000s = df_orig['T6000'].values
        new_rows = events_df[~events_df['T6000'].isin(existing_T6000s)].copy()

        # Fill in missing columns to match structure
        new_rows['reachInit'] = np.nan
        new_rows['reachMax'] = np.nan
        new_rows['reachEnd'] = np.nan
        new_rows['stim'] = 0
        new_rows['behaviors'] = np.nan

        # # Reorder columns to match original file
        # new_rows = new_rows[['reachinit', 'reachmax', 'reachend', 'stim',
        #                      'pellet_delivery', 'pellet_detected', 't6000', 't5000', 'behaviors']]

        # Reorder columns to match original file
        new_rows = new_rows[['T6000', 'T5000', 'reachInit', 'reachMax',
                            'reachEnd', 'stim','behaviors' ,'pellet_delivery', 'pellet_detected']]

        # Combine and sort by T6000
        combined = pd.concat([df_orig, new_rows], ignore_index=True)
        combined = combined.sort_values(by='T6000').reset_index(drop=True)

        # Force final column order
        combined = combined[['T6000', 'T5000', 'reachInit', 'reachMax',
                            'reachEnd', 'stim', 'behaviors',
                            'pellet_delivery', 'pellet_detected']]


        # ---- Step 2: Save output as the same original file (overwrite it)
        xlsx_save_path = xlsx_file
        self.xlsx_file = xlsx_save_path

        original_dir = os.path.dirname(self.xlsx_file)
        backup_dir = os.path.join(original_dir, 'curator_direct_backup')
        backup_file_path = os.path.join(backup_dir, os.path.basename(self.xlsx_file))

        # Save the combined DataFrame to the same xlsx file
        if save_df:
            if os.path.exists(backup_file_path):
                combined.to_excel(xlsx_save_path, index=False)
                print(f"✅ Final merged Excel saved to: {xlsx_save_path}")
                return combined, xlsx_save_path
            else:
                print(f"⚠️Warning: Backup file doesnt exists⚠️")
                print('❌ Stopping the script to avoid overwriting the xlsx file without a back up❌')
                print(f"⚠️Warning: {backup_file_path} does not exist⚠️")
                return None, None
        else:
            print(f"⚠️Warning: DataFrame not saved to {xlsx_save_path}⚠️")
            print('set save_df to True to save the DataFrame')
            return combined
    

    def drop_empty_reach_rows(self):
      
        df = pd.read_excel(self.xlsx_file)
        # back up the newly created xlsx file
        backup_dir_02 = os.path.join(os.path.dirname(self.xlsx_file), 'completed_manual_curation_backup')
        os.makedirs(backup_dir_02, exist_ok=True)
        backup_file_path_02 = os.path.join(backup_dir_02, os.path.basename(self.xlsx_file))
        if os.path.exists(backup_file_path_02):
            print(f"⚠️Warning: Backup file already exists at {backup_file_path_02}⚠️")
            print('❌ Stopping the script to avoid overwriting the backup file ❌')
            return None 
        else:
            # Copy the original file to the backup directory
            shutil.copy2(self.xlsx_file, backup_file_path_02)
            print(f"✅ Backup saved to: {backup_file_path_02}")
        """
        Drop specified rows from the DataFrame.
        """
        # Drop bad rows
        df = df.dropna(subset=['reachInit']).reset_index(drop=True)
         # save the DataFrame to the same xlsx file
        if self.save_df:
            df.to_excel(self.xlsx_file, index=False)
            print(f"✅ Final merged Excel saved to: {self.xlsx_file}")   
              
        return df




In [136]:
safe_fail_modify = False

In [139]:
if safe_fail_modify:
    mouse_sessions = modify_curator_xlsx_file(xlsx_file, txt_file,save_df=True)
    df_modied, xlsx_saved_file = mouse_sessions.create_new_xlxs_file()
    df_modied
else:
    print(f"⚠️Warning: safe_fail actviated⚠️")
    print('if you want to run the code, please re-run this cell')
    print('Note this cell is to prevent accidental overwriting of the xlsx file')
    safe_fail_modify = True

✅ Final merged Excel saved to: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250421_christielab_session001.xlsx


In [140]:
# Load existing curated xlsx
df_modified = pd.read_excel(xlsx_saved_file)
df_modified.head(5)

,T6000,T5000,reachInit,reachMax,reachEnd,stim,behaviors,pellet_delivery,pellet_detected
0,1768,2205.0,NaN,NaN,NaN,0,NaN,2203.0,1823.0
1,4255,4388.0,4423.0,4436.0,4449.0,0,grabbed,4385.0,4308.0
2,6582,6833.0,NaN,NaN,NaN,0,NaN,6831.0,6642.0
3,8895,9263.0,NaN,NaN,NaN,0,NaN,9261.0,8956.0
4,13422,13632.0,13669.0,13681.0,13696.0,0,grabbed,13630.0,13477.0


## Drop empty rows
- this is for after you finished manually curating the behavior session
- there is left over rows with no reachInit values
- the cells bellow does the following:
- 1. duplicates the manually curated .xlsx
  2. drops empty reach rows
  3. saves new .xlsx file back to original location

In [141]:
# set the session info
session_info = '20250421_session001'

In [142]:
if session_info == '20250421_session001':
    # create a new class instance
    reach10_20250421_session01 = select_mouse_session(mouse='reach10', session_date='20250421', session_ID='session001')
    xlsx_file, txt_file, xlsx_orig = reach10_20250421_session01.get_mouse_session_files()
    mouse_info = reach10_20250421_session01.get_mouse_info()
    print(f'mouse_info: {mouse_info}')
    print(f'xlsx_file: {xlsx_file}')

if session_info == '20250424_session001':
    # create a new class instance
    reach10_20250425_session01 = select_mouse_session(mouse='reach10', session_date='20250424', session_ID='session001')
    xlsx_file, txt_file, xlsx_orig = reach10_20250425_session01.get_mouse_session_files()
    mouse_info = reach10_20250425_session01.get_mouse_info()
    print(f'mouse_info: {mouse_info}')
    print(f'xlsx_file: {xlsx_file}')

mouse_info: {'mouse': 'reach10', 'session_date': '20250421', 'session_ID': 'session001'}
xlsx_file: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250421_christielab_session001.xlsx


In [143]:
safe_fail = False

In [145]:
if safe_fail:
    mouse_sessions = modify_curator_xlsx_file(xlsx_file, txt_file,save_df=True)
    df_dropped = mouse_sessions.drop_empty_reach_rows()
    df_dropped
else:
    safe_fail = True
    print(f"⚠️Warning: safe_fail actviated⚠️")
    print('if you want to run the code, please re-run this cell')

✅ Backup saved to: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\completed_manual_curation_backup\20250421_christielab_session001.xlsx
✅ Final merged Excel saved to: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250421_christielab_session001.xlsx
